In [1]:
import pandas as pd
import os

from programs import LongSimulator

In [2]:
TRUCKS_PATH = 'trucks.csv'
ITERATIONS = 1000

In [3]:
def pretty_day(day_idx):
    return {
        0: 'Lunes',
        1: 'Martes',
        2: 'Miércoles',
        3: 'Jueves',
        4: 'Viernes'
    }[day_idx]

def simulate_day(routes_path, day):
    simulator = LongSimulator(str(day), routes_path, TRUCKS_PATH, iterations=ITERATIONS)
    df = pd.DataFrame(simulator.run()).drop('simulation_id', axis=1)
    df['day'] = pretty_day(day)
    df['day_id'] = day
    return df


grouped = ['day_id', 'day', 'truck']

def simulate_days(routes_path):
    df = pd.DataFrame()
    
    for i in range(5):
        df = df.append(simulate_day(routes_path, i))

    return df
    
def get_summaries(df):
    final_df = pd.DataFrame()
    groupby = df.groupby(grouped)
    
    for column in filter(lambda col: col not in grouped, df.columns):
        aux_df = pd.DataFrame()
        
        aux_df['mean'] = groupby[column].mean()
        aux_df['std'] = groupby[column].std()
        aux_df['kpi'] = column
        aux_df = aux_df.reset_index()
        
        final_df = final_df.append(aux_df)
        
    final_df = final_df.set_index(['kpi', *grouped]).sort_index()
    return final_df.reset_index(level=1, drop=True)

def get_trips(df):
    groupby = df.groupby(grouped)

    final_df = pd.DataFrame()
    final_df['mean'] = groupby['trips'].mean()
    final_df['std'] = groupby['trips'].std()
    final_df = final_df.reset_index()

    final_df = final_df.set_index(grouped).sort_index()
    return final_df.reset_index(level=1, drop=True)

In [4]:
outdirs = ['./results/original', './results/optimal']

for outdir in outdirs:
    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [5]:
original = simulate_days('original_routes.csv')

get_summaries(original).to_csv('./results/original/simulation_summary.csv')
get_trips(original).to_csv('./results/original/trip_summary.csv')

In [6]:
optimal = simulate_days('optimal_routes.csv')

get_summaries(optimal).to_csv('./results/optimal/simulation_summary.csv')
get_trips(optimal).to_csv('./results/optimal/trip_summary.csv')